In [51]:
##### ETL NOTEBOOK FOR 2023 MHSAA TOURNEY SPECIFIC MAP

#### Adapted from ETL for JSON

## Dependencies and Setup
### Dependencies

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
import re
import time

## Start timer
start_time = time.time()


In [52]:
## LOAD BLOCK###
#### Load data from kml file exported from Google Earth

file_path = ('data/kml/conf_tourny.kml') # file path to kml file


# Read the KML file
with open(file_path) as file:
    xml_data = file.read()

# Initialize soup variables for parsing file
soup = BeautifulSoup(xml_data, 'xml')
folders = soup.Document
list = soup.Document.find_all('Folder')

# Create a list to store rows to append to the DataFrame
rows = []

# Loop through the folders and extract the data
for folder in list:
    try:
        field_name = folder.find('name').text
        foul = folder.find_all('coordinates')[0].text
        fop = folder.find_all('coordinates')[1].text
        notes = None

        # Check if there is a description tag, if so, use it for notes
        if folder.find('description') is not None:
            notes = folder.find('description').text

        row = {
            'field': field_name,
            'foul': foul,
            'fop': fop,
            'notes': notes
        }

        rows.append(row)

    except Exception as e:
        # Add name of folder to a list of failed folders
        failed.append(folder.find('name').text)
        print(f"Error processing folder: {folder.find('name').text}. Error message: {str(e)}")

# Convert the list of rows to a DataFrame
df = pd.DataFrame(rows, columns=['field', 'foul', 'fop', 'notes'])



In [53]:
# Clean the new dataframe


# Create a copy of the original DataFrame
df_cleaned = df.copy()

# Remove new line and space characters from coordinates
df_cleaned = df_cleaned.replace(r'\n','', regex=True) 
df_cleaned = df_cleaned.replace(r'\t','', regex=True) 

# Drop any duplicate rows
df_cleaned = df_cleaned.drop_duplicates(subset=['field'], keep='first')

# Drop any rows with empty fields
df_cleaned = df_cleaned[(df_cleaned != 0).all(1)]

In [54]:
##### Clean up polygon data and create a new home_plate column

def parse_coordinates(coord_string):
    coords = coord_string.split()
    parsed_coords = [tuple(map(float, coord.split(',')[:2])) for coord in coords]
    return parsed_coords

# Create a new column for the home_plate location using the first set of coordinates in the 'fop' column
df_cleaned['home_plate'] = df_cleaned['fop'].apply(lambda x: parse_coordinates(x)[0])

# Apply the parse_coordinates function to the 'foul' and 'fop' columns
df_cleaned['foul'] = df_cleaned['foul'].apply(parse_coordinates)
df_cleaned['fop'] = df_cleaned['fop'].apply(parse_coordinates)

In [55]:
############## AREA CALCULATION ##############


import pyproj
from shapely.geometry import Polygon
from shapely.ops import transform


def calculate_area(coords):
    # Create a Polygon object from the coordinates
    polygon = Polygon(coords)

    # Calculate the centroid of the polygon
    centroid = polygon.centroid

    # Create a custom LAEA projection centered on the centroid
    custom_projection = f"+proj=laea +lat_0={centroid.y} +lon_0={centroid.x} +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs"

    # Create a transformer for converting coordinates to the custom LAEA projection
    transformer = pyproj.Transformer.from_crs(
        pyproj.CRS("EPSG:4326"),  # WGS 84 (latitude and longitude)
        pyproj.CRS(custom_projection),  # Custom LAEA projection
        always_xy=True
    )

    # Define a function to transform coordinates using the transformer
    def transform_coordinates(x, y):
        return transformer.transform(x, y)

    # Convert the coordinates to the custom LAEA projection
    polygon_laea = transform(transform_coordinates, polygon)

    # Calculate the area in square meters
    area_sqm = polygon_laea.area

    # Convert the area to square feet (1 square meter = 10.764 square feet)
    area_sqft = area_sqm * 10.764

    return area_sqft



### Call Function and add to dataframe
df_cleaned['foul_area_sqft'] = df_cleaned['foul'].apply(calculate_area)
df_cleaned['fop_area_sqft'] = df_cleaned['fop'].apply(calculate_area)

## Calculate the total area of the field and the ratio of foul area to field area
df_cleaned['field_area_sqft'] = df_cleaned['foul_area_sqft'] + df_cleaned['fop_area_sqft']
## Percentage foul area
df_cleaned['foul_area_per'] = df_cleaned['foul_area_sqft'] / df_cleaned['field_area_sqft']
## Fair to Foul Ratio
df_cleaned['fair_to_foul'] = df_cleaned['fop_area_sqft'] / df_cleaned['foul_area_sqft']


In [56]:
############# FENCE DISTANCE CALCULATION #############

from geopy.distance import great_circle
import numpy as np

def interpolate_points(start, end, length_ratio):
    start_np = np.array(start)
    end_np = np.array(end)
    return tuple(start_np + (end_np - start_np) * length_ratio)

def calculate_distances(home_plate, outfield_coords, num_points=540):
    def is_same_point(point1, point2, tolerance=1e-6):
        return abs(point1[0] - point2[0]) < tolerance and abs(point1[1] - point2[1]) < tolerance

    home_plate_lat_lon = (home_plate[1], home_plate[0])
    distances = []

    # Calculate total line length
    total_length = 0
    segments = []
    for i in range(len(outfield_coords) - 1):
        start = outfield_coords[i]
        end = outfield_coords[i + 1]
        if not is_same_point(home_plate, start) and not is_same_point(home_plate, end):
            segment_length = great_circle((start[1], start[0]), (end[1], end[0])).feet
            segments.append((start, end, segment_length))
            total_length += segment_length

    # Calculate the distance between equally spaced points
    spacing = total_length / (num_points - 1)

    # Interpolate points and calculate distances
    current_length = 0
    segment_index = 0
    for i in range(num_points):
        while segment_index < len(segments) - 1 and current_length > segments[segment_index][2]:
            current_length -= segments[segment_index][2]
            segment_index += 1

        start, end, segment_length = segments[segment_index]
        length_ratio = current_length / segment_length
        point = interpolate_points(start, end, length_ratio)
        distance = round(great_circle(home_plate_lat_lon, (point[1], point[0])).feet)
        distances.append(distance)

        current_length += spacing

    return distances

# Calculate distances for each row
df_cleaned['distances'] = df_cleaned.apply(lambda row: calculate_distances(row['home_plate'], row['fop']), axis=1)

# Calculate max, min, and average distances for each row
df_cleaned['max_distance'] = df_cleaned['distances'].apply(max)
df_cleaned['min_distance'] = df_cleaned['distances'].apply(min)
df_cleaned['avg_distance'] = df_cleaned['distances'].apply(lambda distances: sum(distances) / len(distances))
# get the median distance
df_cleaned['median_distance'] = df_cleaned['distances'].apply(lambda distances: np.median(distances))

In [57]:
## Function to create ranks for each column

def rank_fields(df):
    # Calculate the rank for each category
    df['max_distance_rank'] = df['max_distance'].rank(ascending=False, method='min')
    df['min_distance_rank'] = df['min_distance'].rank(ascending=False, method='min')
    df['avg_distance_rank'] = df['avg_distance'].rank(ascending=False, method='min')
    df['median_distance_rank'] = df['median_distance'].rank(ascending=False, method='min')
    df['field_area_rank'] = df['field_area_sqft'].rank(ascending=False, method='min')
    df['foul_area_rank'] = df['foul_area_sqft'].rank(ascending=False, method='min')
    df['fop_area_per_rank'] = df['fop_area_sqft'].rank(ascending=False, method='min')
    df['ratio_rank'] = df['fair_to_foul'].rank(ascending=False, method='min')

    return df

## Run Function

df_cleaned = rank_fields(df_cleaned)



In [58]:
#### Orienting the map to the home plate location ####

### Find the center of the field
def calculate_centroid(coords):
    x_coords = [coord[0] for coord in coords]
    y_coords = [coord[1] for coord in coords]
    centroid_x = sum(x_coords) / len(coords)
    centroid_y = sum(y_coords) / len(coords)
    return (centroid_x, centroid_y)


## Find the bearing between the home plate and the center of the field
import math

def calculate_bearing(point1, point2):
    lat1, lon1 = math.radians(point1[1]), math.radians(point1[0])
    lat2, lon2 = math.radians(point2[1]), math.radians(point2[0])

    d_lon = lon2 - lon1

    x = math.cos(lat2) * math.sin(d_lon)
    y = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(d_lon)

    bearing = math.degrees(math.atan2(x, y))
    bearing = (bearing + 360) % 360  # Normalize the bearing to the range [0, 360)

    return bearing

### Function to classify direction in laymans terms North, South, East, West, ect
def degrees_to_cardinal_direction(degrees):
    directions = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW', 'N']
    index = round(degrees / 45)
    return directions[index]

In [59]:
# Calculate the centroid of the outfield fence coordinates for each row
df_cleaned['fop_centroid'] = df_cleaned['fop'].apply(lambda coords: calculate_centroid(coords[1:]))

# Calculate the bearing between home plate and the centroid for each row
df_cleaned['field_orientation'] = df_cleaned.apply(lambda row: calculate_bearing(row['home_plate'], row['fop_centroid']), axis=1)

# Convert the bearing to a cardinal direction
df_cleaned['field_cardinal_direction'] = df_cleaned['field_orientation'].apply(degrees_to_cardinal_direction)

# rename 'field' to 'park_name'
df_cleaned.rename(columns={'field': 'park_name'}, inplace=True)

In [60]:
### THIS BLOCK CREATES THE RANKING OF PITCHER VS HITTER FRIENDLY FIELDS
def rank_fields(data):
    # Define weights for each parameter
    weights = {
        'max_distance': -1, # negative weight since longer fences favor pitchers
        'min_distance': 1,  # positive weight since shorter fences favor hitters
        'avg_distance': -1, # negative weight since longer fences favor pitchers
        'median_distance': -1, # negative weight since longer fences favor pitchers
        'field_area_sqft': -1,  # negative weight since larger fields favor pitchers
        'fair_to_foul': -1,  # negative weight since larger ratio (more foul territory) favors pitchers
        'foul_area_sqft': -1, # negative weight since larger foul area favors pitchers
        'fop_area_sqft': -1, # negative weight since larger out of play area favors pitchers
    }

    # Standardize features (subtract mean and divide by standard deviation)
    standardized_data = data.copy()
    for column in weights.keys():
        standardized_data[column] = (standardized_data[column] - standardized_data[column].mean()) / standardized_data[column].std()

    # Calculate score for each field
    standardized_data['score'] = standardized_data.apply(lambda row: sum(row[param] * weight for param, weight in weights.items()), axis=1)

    # Save scores to original dataframe
    data['score'] = standardized_data['score']

    # Rank fields based on score (higher scores are more hitter-friendly)
    ranked_fields = data.sort_values('score', ascending=False)

    return ranked_fields

# Suppose 'df' is your DataFrame containing the field data
df = rank_fields(df_cleaned)
print(df[['park_name', 'score']])

                                 park_name      score
12                   Russ Chandler Stadium   7.992146
15  Binghamton University Baseball Stadium   6.134127
18              Durham Bulls Athletic Park   5.035419
25                     Bob Bennett Stadium   3.848177
17                       The Diamond - VCU   3.326890
11                     Joe Miller Ballpark   3.166903
16                        BayCare Ballpark   2.443897
5       Bob Warn Field at Sycamore Stadium   2.370159
10             Fluor Field Greenville, S.C   1.799821
2   Olga Mural Field at Schoonover Stadium   1.596515
20                             Prasco Park   0.963038
3       Johnson Stadium at Doubleday Field   0.735563
21                            Truist Point   0.201779
23                           Reckling Park   0.133616
13                       Riverwalk Stadium   0.004898
24                          Clover Stadium  -0.135177
19                             Conrad Park  -0.872847
14                      Las 

## Working to here: fields are loaded and calcs are done. Need to merge in data after

In [61]:
### Merge in the Name of the conference
df['park_name'].tolist()

['Russ Chandler Stadium',
 'Binghamton University Baseball Stadium',
 'Durham Bulls Athletic Park',
 'Bob Bennett Stadium',
 'The Diamond - VCU',
 'Joe Miller Ballpark',
 'BayCare Ballpark',
 'Bob Warn Field at Sycamore Stadium',
 'Fluor Field Greenville, S.C',
 'Olga Mural Field at Schoonover Stadium',
 'Prasco Park',
 'Johnson Stadium at Doubleday Field',
 'Truist Point',
 'Reckling Park',
 'Riverwalk Stadium',
 'Clover Stadium',
 'Conrad Park',
 'Las Vegas Ballpark',
 'Heritage Financial Park',
 'Charles Schwab Field',
 'Marion Park - Mt Dew Park',
 'Hoover Metropolitan Stadium',
 'Nischwitz Stadium',
 'Hohokam Stadium',
 'The Ballpark at Patriots Point',
 'Scottsdale Stadium']

In [62]:
## Open the CSV file with the conference names
conf_df = pd.read_csv('data/NCAA_D1/site_table.csv')

df.info()
conf_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 12 to 8
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   park_name                 26 non-null     object 
 1   foul                      26 non-null     object 
 2   fop                       26 non-null     object 
 3   notes                     0 non-null      object 
 4   home_plate                26 non-null     object 
 5   foul_area_sqft            26 non-null     float64
 6   fop_area_sqft             26 non-null     float64
 7   field_area_sqft           26 non-null     float64
 8   foul_area_per             26 non-null     float64
 9   fair_to_foul              26 non-null     float64
 10  distances                 26 non-null     object 
 11  max_distance              26 non-null     int64  
 12  min_distance              26 non-null     int64  
 13  avg_distance              26 non-null     float64
 14  median_dista

In [63]:
## Merge the two dataframes

df = pd.merge(df, conf_df, on='park_name', how='outer')

df.info()
# df.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   park_name                 27 non-null     object 
 1   foul                      26 non-null     object 
 2   fop                       26 non-null     object 
 3   notes                     0 non-null      object 
 4   home_plate                26 non-null     object 
 5   foul_area_sqft            26 non-null     float64
 6   fop_area_sqft             26 non-null     float64
 7   field_area_sqft           26 non-null     float64
 8   foul_area_per             26 non-null     float64
 9   fair_to_foul              26 non-null     float64
 10  distances                 26 non-null     object 
 11  max_distance              26 non-null     float64
 12  min_distance              26 non-null     float64
 13  avg_distance              26 non-null     float64
 14  median_dista

In [64]:
## Drop rows that don't have any field coordinate data because that breaks the map
df = df.dropna(subset=['foul', 'fop', 'home_plate'])

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 0 to 25
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   park_name                 26 non-null     object 
 1   foul                      26 non-null     object 
 2   fop                       26 non-null     object 
 3   notes                     0 non-null      object 
 4   home_plate                26 non-null     object 
 5   foul_area_sqft            26 non-null     float64
 6   fop_area_sqft             26 non-null     float64
 7   field_area_sqft           26 non-null     float64
 8   foul_area_per             26 non-null     float64
 9   fair_to_foul              26 non-null     float64
 10  distances                 26 non-null     object 
 11  max_distance              26 non-null     float64
 12  min_distance              26 non-null     float64
 13  avg_distance              26 non-null     float64
 14  median_dista

In [66]:
## print all home_plates 

In [67]:
### Output the Files as a JSON and as a csv for review

df.to_csv('data/NCAA_D1/conf_tourn_map.csv', index=False)

df.to_json('data/NCAA_D1/conf_tourn_map2.json', orient='records')